# NN Feature Extraction -> Gradient Boosting

In [72]:
import sys
# sys.path.append('..')
from pytorch_utils import *
from lightning_utils import *
from pytorch_vision_utils import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [73]:
data_path = r'E:\Data_and_Models\Kaggle_Cards'
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Import a NN for feature extraction

In [74]:
# Import the feature-extraction portion of the models

classes = [os.path.basename(p) for p in Path(fr'{data_path}\test').glob('*')]

# The below timm.create_model calls support the features_only = True argument, but that stops before the final convolution before the classification layer,
#   thus (for RexNet 1.0) outputting 5 tensors of (squeezed) shapes
#       torch.Size([16, 112, 112])
#       torch.Size([38, 56, 56])
#       torch.Size([61, 28, 28])
#       torch.Size([128, 14, 14])
#       torch.Size([185, 7, 7])
#   instead of the single one going into classification ([1280, 7, 7]), which is then pooled to 1280 features in the first part of the head portion (1920 for RexNet 1.5).

# model_name = 'RexNet10'
# model = timm.create_model('rexnet_100.nav_in1k', pretrained = True, num_classes = 53).eval().to(device)

model_name = 'RexNet15'
model = timm.create_model('rexnet_150.nav_in1k', pretrained = True, num_classes = 53).eval().to(device)


transforms = timm.data.create_transform(**timm.data.resolve_model_data_config(model), is_training = False)

# No training and only feature extraction (up to pooling after final convolution, i.e. for RexNet 1.0 [batch, 1280, 7, 7] -> [batch, 1280], and 1920 for RexNet 1.5)
for param in model.parameters(): param.requires_grad = False
model = nn.Sequential(OrderedDict(stem = model.stem, features = model.features, pool = model.head.global_pool))

In [75]:
summ(model, input_size = (32, 3, 224, 224))

Layer (type (var_name))                       Input Shape          Output Shape         Param #              Trainable
Sequential (Sequential)                       [32, 3, 224, 224]    [32, 1920]           --                   False
├─ConvNormAct (stem)                          [32, 3, 224, 224]    [32, 48, 112, 112]   --                   False
│    └─Conv2d (conv)                          [32, 3, 224, 224]    [32, 48, 112, 112]   (1,296)              False
│    └─BatchNormAct2d (bn)                    [32, 48, 112, 112]   [32, 48, 112, 112]   96                   False
│    │    └─Identity (drop)                   [32, 48, 112, 112]   [32, 48, 112, 112]   --                   --
│    │    └─SiLU (act)                        [32, 48, 112, 112]   [32, 48, 112, 112]   --                   --
├─Sequential (features)                       [32, 48, 112, 112]   [32, 1920, 7, 7]     --                   False
│    └─LinearBottleneck (0)                   [32, 48, 112, 112]   [32, 24, 112, 1

## Extract features

In [76]:
def images_to_features(image_folder: str | Path, model: torch.nn.Module, transform: tv.transforms.Compose, class_names: list[str],
                       device: torch.device = 'cuda' if torch.cuda.is_available() else 'cpu'):
    image_folder = Path(image_folder)
    true_classes, features = [], []

    image_paths = list(image_folder.glob('*/*.jpg')) + list(image_folder.glob('*/*.png'))

    # Should not trust e.g. sklearn.preprocessing.LabelEncode to assign class indices later, hence doing it here
    class_map = {v: i for i, v in enumerate(classes)}

    for path in tqdm(image_paths, desc = f'Extracting {image_folder.name} images\' features'):
        img = Image.open(path)

        model.eval()
        with torch.inference_mode(): feats = model(transform(img).unsqueeze(0).to(device)).squeeze().to('cpu')

        true_classes.append(class_map[path.parent.stem])
        features.append(feats)
    
    return np.array(features), np.array(true_classes)

In [77]:
train_features, train_classes = images_to_features(image_folder = fr'{data_path}\train', model = model, transform = transforms, class_names = classes)
valid_features, valid_classes = images_to_features(image_folder = fr'{data_path}\valid', model = model, transform = transforms, class_names = classes)

Extracting train images' features:   0%|          | 0/7624 [00:00<?, ?it/s]

Extracting valid images' features:   0%|          | 0/265 [00:00<?, ?it/s]

In [78]:
valid_features.shape, valid_classes.shape

((265, 1920), (265,))

## Gradient Boosting

In [79]:
import lightgbm as lgb
from sklearn.metrics import confusion_matrix

In [80]:
# model_name = 'RexNet10'
model_name = 'RexNet15'

# lgb_train = lgb.Dataset(train_features, label = train_classes, params = dict(max_bin = 63)) # max_bin is recommended smaller than 255 for gpu and cuda
# lgb_valid = lgb_train.create_valid(valid_features, label = valid_classes, params = dict(max_bin = 63))

# lgb_train.save_binary(f'{data_path}\{model_name}_features_training_data.bin')
# lgb_valid.save_binary(f'{data_path}\{model_name}_features_validation_data.bin')

lgb_train = lgb.Dataset(f'{data_path}\{model_name}_features_training_data.bin')
lgb_valid = lgb.Dataset(f'{data_path}\{model_name}_features_validation_data.bin')

[LightGBM] [Info] Saving data to binary file E:\Data_and_Models\Kaggle_Cards\RexNet15_features_training_data.bin
[LightGBM] [Info] Saving data to binary file E:\Data_and_Models\Kaggle_Cards\RexNet15_features_validation_data.bin


In [81]:
num_iterations = 100
boosting_type = 'gbdt' # 'gbdt' vs 'dart' (dart also comes with more parameters: max_drop, skip_drop, xgboost_dart_mode, uniform_drop)
data_sample_strategy = 'bagging' # 'bagging' vs 'goss' (goss also comes with more parameters: top_rate, other_rate)

# VERY extensive set of parameters to play with: https://lightgbm.readthedocs.io/en/stable/Parameters.html
lgb_params = dict(device = 'gpu', max_bin = 63, # Recommended smaller than 255 for gpu and cuda
    boosting_type = boosting_type, data_sample_strategy = data_sample_strategy,
    objective = 'multiclass', metric = 'multi_logloss', num_class = len(classes),
    # objective = 'cross_entropy', metric = 'cross_entropy',# num_class = 1,
    num_iterations = num_iterations,
    learning_rate = 0.05,
    # num_leaves = 31, max_depth = 10,
    # feature_fraction = 0.7, bagging_fraction = 0.8, bagging_freq = 5,
    # early_stopping_round = 3,
    # lambda_l1 = 0.0, lambda_l2 = 0.0, # Regularisation is also an option
    seed = 42)

In [82]:
gb_model = lgb.train(lgb_params, lgb_train, valid_sets = lgb_valid) # , callbacks=[lgb.early_stopping(stopping_rounds = 5) # unclear precedence with early_stopping_round parameter

c:\Users\Dr-Lo\miniconda3\envs\ML11\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Load from binary file E:\Data_and_Models\Kaggle_Cards\RexNet15_features_training_data.bin
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 120960
[LightGBM] [Info] Number of data points in the train set: 7624, number of used features: 1920
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce GTX 970, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 64 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 1920 dense feature groups (13.96 MB) transferred to GPU in 0.023837 secs. 0 sparse feature groups
[LightGBM] [Info] Start training from score -4.151565
[LightGBM] [Info] Start training from score -4.079244
[LightGBM] [Info] Start training from score -3.797393
[LightGBM] [Info] Start training from score -3.740559
[LightGBM] [Info] Start training from score -4.011803
[LightGBM] [Info] Start training from score -3.870152
[LightGBM] [Info] Start training from

In [83]:
gb_model.save_model(fr'{data_path}\Models\{model_name}_features_in_{num_iterations}_{data_sample_strategy}_{boosting_type}_lgbm.txt')

## Testing

In [84]:
# boosting_type = 'gbdt' # 'gbdt' vs 'dart'
# data_sample_strategy = 'bagging' # 'bagging' vs 'goss'
gb_model = lgb.Booster(model_file = fr'{data_path}\Models\{model_name}_features_in_{num_iterations}_{data_sample_strategy}_{boosting_type}_lgbm.txt') 

In [85]:
test_features, test_classes = images_to_features(image_folder = fr'{data_path}\test', model = model, transform = transforms, class_names = classes)

Extracting test images' features:   0%|          | 0/265 [00:00<?, ?it/s]

In [86]:
logits = gb_model.predict(test_features, num_iteration = gb_model.best_iteration)
pred_classes = torch.argmax(torch.softmax(torch.from_numpy(logits), dim = 1), dim = 1)

In [87]:
f1_fn = torchmetrics.F1Score(task = 'multiclass', num_classes = 53)
f1_fn(pred_classes, torch.from_numpy(test_classes)).item()

0.5433962345123291

### Confusion Matrix

In [88]:
conf_mat = confusion_matrix(test_classes, pred_classes)
conf_df = pd.DataFrame([(classes[real], classes[pred], conf_mat[real, pred]) for real in range(len(classes)) for pred in range(len(classes))], columns = ['Real', 'Pred', 'Count'])
conf_df

,Real,Pred,Count
0,ace of clubs,ace of clubs,4
1,ace of clubs,ace of diamonds,0
2,ace of clubs,ace of hearts,0
3,ace of clubs,ace of spades,0
4,ace of clubs,eight of clubs,0
...,...,...,...
2804,two of spades,three of spades,1
2805,two of spades,two of clubs,0
2806,two of spades,two of diamonds,0
2807,two of spades,two of hearts,0


In [89]:
# Confusion Matrix

alt.Chart(conf_df).mark_rect().encode(
    x = 'Pred:O', y = 'Real:O',
    color = alt.Color('Count:Q', scale = alt.Scale(scheme = 'blues'), legend = alt.Legend(title = 'Count'))
).properties(
    title = 'Confusion Matrix'
)

alt.Chart(...)

# Feature Importance

In [90]:
feats_by_importance = pd.DataFrame(dict(feature = gb_model.feature_name(), importance = gb_model.feature_importance())).sort_values('importance', ascending = False)
feats_by_importance

,feature,importance
1141,Column_1141,627
294,Column_294,582
945,Column_945,456
683,Column_683,377
853,Column_853,336
...,...,...
1371,Column_1371,25
1430,Column_1430,24
1461,Column_1461,22
1235,Column_1235,17
